## Homicidios
### Source: https://www.ipea.gov.br/atlasviolencia/filtros-series

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from dtaidistance.dtw import distance_matrix, distance
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import picos as pic
import itertools as it
import copy
import json

In [2]:
#N = number of neighbors - 1 up to 20
#feature = path of feature file
#N=5
#feature = pd.read_csv("data/IBGE/census_2010/RS/aggregated_by_weighting_area/joined/Renda.csv",index_col=0)
def graph_feat(N,feature,feature_col,file_matrix,file_hom):
    # constrói um grafo knn 

    adj = pd.read_csv(file_matrix,index_col=0)
    adj.index = adj.index.map(str)
    
    def build_graph(adj,nn_radi):
        names = adj.index
        G = nx.Graph()
        tmp = np.array(adj)
        tmp = np.argsort(tmp)
        tmp = np.argwhere(tmp < nn_radi)
        for e in tmp:
            #if para testar se e[0]electionwhowon != e[1]electionwhowon
            G.add_edge(names[e[0]],names[e[1]])
        return(G)

    # grafos de 1 até N vizinhos

    G = [build_graph(adj,i) for i in range(1,N)]
    
    feature.index = feature.index.map(str)
    #print(feature.index)
    #print("feature.columns",feature.columns)

    for gg in G:
        attrs={}

        for edge in gg.edges():
            attrs[(edge[0],edge[1])] = {}

            for v in feature.columns[feature_col:]:

                #se quiser trocar np.abs para calcular a diferença
                #w = np.abs(feature[v][edge[0]] - feature[v][edge[1]]) 
                w = np.mean([feature[v][edge[0]] , feature[v][edge[1]]])       

                attrs[(edge[0],edge[1])][v] = w
        
        nx.set_edge_attributes(gg, attrs)
    
    hom = pd.read_csv(file_hom,index_col=0)
    hom.index = hom.index.map(str)
    
    for gg in G:
        attrs={}

        for edge in gg.edges():
            try:
                b0 = hom['homicidios'][edge[0]]
                b1 = hom['homicidios'][edge[1]]

                attrs[(edge[0],edge[1])] = {'hom' : np.mean([b0,b1])}
                #attrs[(edge[0],edge[1])] = {'hom' : np.abs([b0-b1])}
                                            
            except:
                pass

        nx.set_edge_attributes(gg, attrs)
    
    ### I.Preprocessing Phase: Computations over the Domain

    K = len(G)

    #Amount of features
    F= len(feature.columns[feature_col:])

    def feat_edge(D,x):
        data_2 = [v for _,_,v in D.edges.data(x)]
        data_2_at = [v for v in data_2 if v is not None]
        d2 = np.histogram(data_2_at,bins=5)[0]
        #fixed the number of bins, not using scott anymore
        #d2 = np.histogram(data_2_at,bins=5)[0]
        return(d2/float(sum(d2)))

    ### Step 1: Feature Representation Model


    #for i in range(K):
        #PDF computed for edges
        #PDF_e[i] = feat_edge((D[i]),'num_critic_for_reviews'),feat_edge((D[i]),'duration'),feat_edge((D[i]),'director_facebook_likes'), feat_edge((D[i]),'actor_3_facebook_likes'), feat_edge((D[i]),'actor_1_facebook_likes'),feat_edge((D[i]),'gross'),feat_edge((D[i]),'num_voted_users'),feat_edge((D[i]),'cast_total_facebook_likes'),feat_edge((D[i]),'facenumber_in_poster'), feat_edge((D[i]),'num_user_for_reviews'), feat_edge((D[i]),'budget'), feat_edge((D[i]),'title_year'),feat_edge((D[i]),'actor_2_facebook_likes'),feat_edge((D[i]),'movie_facebook_likes')

    PDF_e = [[feat_edge(gg,v) for v in feature.columns[feature_col:]] for gg in G]

    #for i in range(K): 
    #    PDF_e.append([])
    #    for j,v in enumerate(alfabetizacao.columns[2:]):
    #        PDF_e[i].append(feat_edge(G[i],v)) 
    
    ### Step 2: Feature Diversity Model

    dists_e = [distance_matrix(PDF_e[i]) for i in range(K)]
    
    #edge
    SF_e = np.nan_to_num(np.mean(dists_e,axis=0),posinf=0)
    
    
    ### II. Query Phase: Summary Creation

    ### Step 1: Domain-specificity Model
    PDF_ge = [feat_edge(gg,'hom') for gg in G]
    
    
    diff_e = [[distance(PDF_ge[i],PDF_e[i][l]) for l in range(F)] for i in range(K)]

    h_e= np.mean(diff_e,axis=0)
                                                                                 
    ### Step 2: Feature Selection

    #Regularization Parameters l1,l2, l3

    l1 = 1/F
    l2 = 1
    l3 = 1

    #Matriz identidade: np.identity(F)

    Q_e = np.multiply(SF_e,l1) + np.multiply(np.identity(F),l2)
    r_e = np.multiply(h_e,l3)

    sel_feat = 3

    #Solver Edge
    prob_e = pic.Problem()
    K=F
    #f = pic.IntegerVariable('f',K)

    f = pic.RealVariable('f',K)
    prob_e.add_constraint(pic.sum(f.T)<=F)
    #prob_e.add_constraint(pic.sum(f.T)<F)
    prob_e.add_constraint(pic.sum(f.T)>=sel_feat)
    #Q_e = pic.new_param('Q',Q_e)
    #r_e = pic.new_param('r',r_e)
    prob_e.set_objective('min',f.T*Q_e*f+f.T*r_e)
    sol_e = prob_e.solve(solver="ecos",primals=None,duals=None,max_footprints=100,verbosity=0,rel_prim_fsb_tol=1e-3,rel_dual_fsb_tol=1e-3,abs_prim_fsb_tol=1e-3,abs_dual_fsb_tol=1e-3)

    values = np.array(list(sol_e.primals.values())[0])
    
    #order = np.argsort(-v)
    order = np.argsort(-values)

    result_order = list(zip(feature.columns[feature_col:][order],values[order]))
    
    features_order = list(feature.columns[feature_col:][order])
    
    y = np.round(list(sol_e.primals.values()))
    result = np.where(y == 1)
    
    
    return(features_order,result_order, result)



In [3]:
#Numero de vizinhos
N=2
#Coluna com o Codigo do municipio (as features são consideradas a partir desta coluna)
feature_col = 20
feature = pd.read_csv("data/Brasil_new/data_noelections.csv", index_col=feature_col)
features_order,result_order, result = graph_feat(N,feature,feature_col,"data/Brasil_new/filtered_adjacency_matrix_inv.csv","data/homicidios/homicidios_2010_abs.csv")


/Users/greta/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = pd.DataFrame.from_dict({ 'Ordem' : result_order
                  }, orient='index').transpose()

In [5]:
df.head(10)

,Ordem
0,"(CENSUS_PessoaRenda_V121, 0.31601626924161696)"
1,"(CENSUS_PessoaRenda_V055, 0.31592997128330463)"
2,"(CENSUS_PessoaRenda_V077, 0.3159106132008879)"
3,"(CENSUS_PessoaRenda_V011, 0.3159104890451584)"
4,"(CENSUS_ResponsavelRenda_V121, 0.3155686906821..."
5,"(CENSUS_ResponsavelRenda_V055, 0.3155685760037..."
6,"(CENSUS_Pessoa09_V188, 0.31556181671711475)"
7,"(CENSUS_Domicilio01_V181, 0.31549469758082144)"
8,"(CENSUS_ResponsavelRenda_V100, 0.315478894215076)"
9,"(CENSUS_ResponsavelRenda_V034, 0.3154787794944..."


In [6]:
#features_order[1:157]

#data_json = {'selected_features':features_order[1:11]}

data_json = {'selected_features':features_order[0:18]}

data_json

{'selected_features': ['CENSUS_PessoaRenda_V121',
  'CENSUS_PessoaRenda_V055',
  'CENSUS_PessoaRenda_V077',
  'CENSUS_PessoaRenda_V011',
  'CENSUS_ResponsavelRenda_V121',
  'CENSUS_ResponsavelRenda_V055',
  'CENSUS_Pessoa09_V188',
  'CENSUS_Domicilio01_V181',
  'CENSUS_ResponsavelRenda_V100',
  'CENSUS_ResponsavelRenda_V034',
  'CENSUS_ResponsavelRenda_V077',
  'CENSUS_ResponsavelRenda_V011',
  'CENSUS_Entorno05_V1042',
  'CENSUS_Domicilio02_V036',
  'CENSUS_PessoaRenda_V100',
  'CENSUS_PessoaRenda_V034',
  'CENSUS_Entorno05_V1022',
  'CENSUS_Domicilio01_V173']}

In [7]:
def escrever_json(lista):
    with open('resultados/hom_abs_n2_18_avg.json', 'w') as f:
        json.dump(lista, f, indent=4)
    
escrever_json(data_json)

In [ ]:
data_json_full = {'selected_features':features_order[0:]}

def escrever_json_full(lista):
    with open('resultados_full/GRAPH_ELECTIONS_CENSUS_ALL1.json', 'w') as f:
        json.dump(lista, f, indent=4)
    
escrever_json_full(data_json_full)

In [ ]:
data_json_full